# Rigid Body Molecular Dynamics Simulations

## Overview

### Questions

* How do input rigid bodies into hoomd? 
* How do I correctly integrate the rigid bodies in my simulation? 

### Objectives

* Describe how to correctly add a rigid body to a hoomd simulation with the correct physical properties. 
    * Mass 
    * Moment of inertia
* Explain how filtering of the rigid body works in hoomd.
* Explain integration of rotational degrees of freedom.
* Show how hoomd adds constituent particles to a rigid body.

## Boilerplate code

In [1]:
import itertools
import math

import gsd.hoomd
import hoomd
import numpy
import rowan

from hoomd.md import constrain

The `render_rigid_body` function in the next (hidden) cell will render the rigid body using **fresnel**.

<div class="alert alert-info">
    This is not intended as a full tutorial on <b>fresnel</b> - see the <a href="https://fresnel.readthedocs.io/">fresnel user documentation</a> if you would like to learn more.
</div>

In [2]:
import fresnel
import IPython

device = fresnel.Device()
tracer = fresnel.tracer.Path(device=device, w=300, h=300)

def render_rigid_body(pos_body, radius_constituent):
    '''
    outputs the sphere positions in the rigid body depending on the radius of the constituent particles
    '''
    L = 3 * numpy.max(numpy.linalg.norm(pos_body, axis=1))
    scene = fresnel.Scene(device)
    geometry = fresnel.geometry.Sphere(scene,
                                       N=len(pos_body),
                                       radius=radius_constituent)
    geometry.material = fresnel.material.Material(color=fresnel.color.linear(
        [252 / 255, 209 / 255, 1 / 255]),
                                                  roughness=0.5)
    geometry.position[:] = pos_body
    geometry.outline_width = 0.04
    box = fresnel.geometry.Box(scene, [L, L, L, 0, 0, 0], box_radius=.02)
    
    scene.lights = [
        fresnel.light.Light(direction=(0, 0, 1),
                            color=(0.8, 0.8, 0.8),
                            theta=math.pi),
        fresnel.light.Light(direction=(1, 1, 1),
                            color=(1.1, 1.1, 1.1),
                            theta=math.pi / 3)
    ]
    scene.camera = fresnel.camera.orthographic(position=(L * 2, L, L * 2),
                                               look_at=(0, 0, 0),
                                               up=(0, 1, 0),
                                               height=L * 1.4 + 1)
    scene.background_color = (1, 1, 1)
    return IPython.display.Image(tracer.sample(scene, samples=500)._repr_png_())

In the previous section we outlined the rigid body physics of a dumbell particle, in this case two spheres fused together. Below we briefly define the properties of a dumbell where each sphere has a mass of 1 and radius of 1, with a more detailed description in previous section. 

The mass of the rigid body is therefore 2. 

If the spheres have positions $[−1,0,0]$ and $[1,0,0]$ then the moment of inertia tensor is:

\begin{bmatrix}
24/5&0&0\\
0& 14/5& 0\\
0&0&14/5
\end{bmatrix}

In [3]:
Mass_Rigid_body = 2.
Inertia_Tensor = numpy.array([[24/5,0,0],[0,14/5,0],[0,0,14/5]])
local_position = numpy.array([[1,0,0],[-1,0,0]])

# Hoomd's Definition of rigid bodies:

Hoomd defines rigid bodies as a body with a position and orientation in global coordinates. The rigid body is composed of force fields given in the local coordinates of the rigid body, known as *constituent particles*. To integrate the rigid bodies, hoomd sums over the forces of the *constituent particles*.

For this tutorial we will assume the force field for each constituent particle is a Lennard-Jones force field, as each free sphere is in the introduction to molecular dynamics. 

The moment of inertia of a rigid body is defined by an inertia tensor, but it is important to note that in hoomd we assume that the rigid body is oriented such that the moment of inertia tensor is *diagonalized*. In the case of the dumbell above this is already the case and we can put the xx,yy, and zz components of the inertia tensor matrix into hoomd. Section 4 covers how to add bodies to hoomd that are oriented in a way that the inertia tensor is not diagonalized.


# How to integrate the rigid bodies:

Rigid bodies are often anisotropic and their rotational degrees of freedom need to be integrated to evolve their orientation through time. Hoomd requires the user to explicitly set that you want to integrate the rotational degrees of freedom.

In order to only integrate over the rigid bodies and not the constituent particles that make up the body, hoomd uses a filter that tells the integrator to only integrate over the rigid bodies and any other free bodies in the system. This allows the integration to happen for only the rigid bodies. 

We will go through this whole process below, but first, we give a checklist of how to initialize a rigid body.

# Steps to Initializing a rigid body simulation: 

* Rigid Body Initialization 

    * Initialize simulation from a snapshot containing only the positions and properties of the rigid bodies, not the constituent particles. 
    * Create a cell list the filters out the rigid bodies which are composed of constituent particles
    * Create a **filter** for the integrator that only integrates the rigid bodies 
    * Create a force field for the constituent particles
    * Create an integrator
    * Add a rigid body constraint and define the particles within the rigid body
    * Create particles in the rigid body in the simulation. 
    * Add rigid bodies to the integrator

* Final Setup
    * Give particles random momenta drawn correctly drawn from the Boltzmann distribution for the current temperature
    * Run Simulation at Desired Temperature


We will first show how to initialise a snapshot of rigid dumbells below. 

In [4]:
m = 2
N_particles = 2 * m**3
spacing = 10.
K = math.ceil(N_particles**(1 / 3))
L = K * spacing
x = numpy.linspace(-L / 2, L / 2, K, endpoint=False)
position = list(itertools.product(x, repeat=3))
position = numpy.array(position)
position[:,0] = position[:,0]
snapshot = gsd.hoomd.Snapshot()
snapshot.particles.N = N_particles
snapshot.particles.position = position[0:N_particles,:]
snapshot.particles.orientation = [(1,0,0,0)] * N_particles
snapshot.particles.typeid = [0] * N_particles
snapshot.configuration.box = [L, L, L, 0, 0, 0]
snapshot.particles.mass = [Mass_Rigid_body] * N_particles
snapshot.particles.types = ['rigid_body','constituent_sphere']
snapshot.particles.moment_inertia = [(Inertia_Tensor[0,0],Inertia_Tensor[1,1],Inertia_Tensor[2,2])] * N_particles
snapshot.particles.angmom = [(1,1,1,1)] * N_particles
with gsd.hoomd.open(name='lattice_rigid.gsd', mode='wb') as f:
    f.append(snapshot)

# Simulation Parameters:

Like our previous md simulations, we will choose a temperature (1 kT for our system) as and run the system at constant volume. In order to maintain a constant temperature we will use a Nose-Hoover thermostat. 

Notice how we have not yet defined the local coordinates of constituent particles of the rigid body. This is because in hoomd we start with a snapshot of the central coordinates of the rigid body and then add a constraint. In this case we use a rigid body constraint that integrates only over the rigid bodies in the system. 

In [5]:
temperature = 1.

Create a simulation object and device

In [6]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=1)
sim.create_state_from_gsd(filename='lattice_rigid.gsd')

# Parameters for the Lennard-Jones force field

Each sphere that makes up the dumbell interacts with a Lennard Jones potential that has two parameters:
* $\epsilon$ - The prefactor in front of the LJ potential
* $\sigma$ - Dictates the range of the LJ potential

We assume the radius of the particle is the half the minimum distance between two LJ spheres at minimum pairwise energy. 

$r_m = 2^{1/6}\sigma$

We therefore set $r_m$ to 2 and therefore $\sigma$ equals $2/2^{1/6}$

We also assume $\epsilon = 1.$

# Parameters of the Integrator:

An important thing to note is that we must tell hoomd to integrate the rotational degrees of freedom, otherwise our rigid body will not change from the initial orientation. 

In [7]:
epsilon = 1
sigma = 2/2**(1/6)
integrator = hoomd.md.Integrator(dt=0.0001,integrate_rotational_dof=True)
cell = hoomd.md.nlist.Cell(buffer=0,exclusions = ['body'])
lj = hoomd.md.pair.LJ(nlist=cell)
lj.params[('rigid_body', 'rigid_body')] = dict(epsilon=0, sigma=sigma)
lj.params[('rigid_body', 'constituent_sphere')] = dict(epsilon=0, sigma=sigma)
lj.params[('constituent_sphere', 'constituent_sphere')] = dict(epsilon=epsilon, sigma=sigma)
lj.r_cut[('constituent_sphere', 'constituent_sphere')] = 4 * sigma 
lj.r_cut[('rigid_body', 'rigid_body')] = 0.
lj.r_cut[('rigid_body', 'constituent_sphere')] = 0.
integrator.forces.append(lj)

## Adding rigid body constraint:

This adds the rigid bodies to the simulations and marks them as 'body' type particles. 

In [8]:
rigid = constrain.Rigid()

constituent_types = ['constituent_sphere','constituent_sphere']

rigid.body['rigid_body'] = {
    "constituent_types": constituent_types,
    "positions":local_position,
    "orientations":[(1,0,0,0)] * len(constituent_types),
    "charges":[0.] * len(constituent_types),
    "diameters":[2.] * len(constituent_types)}

# Filtering integration in hoomd:

Hoomd must filter out the integration of only the rigid bodies and has a built in method to do so.

We must apply this method in our Nose-Hoover NVT integrator. 

*Center* particles are denoted as the rigid body centers defined above. *free* particles are particles that are neither a rigid body center, nor constituent particles of a rigid body.  

In [9]:
rigid_centers_and_free_filter = hoomd.filter.Rigid(("center", "free"))
nvt = hoomd.md.methods.NVT(kT = temperature, tau = 1., filter = rigid_centers_and_free_filter)
integrator.methods.append(nvt)

## Running a Simulation in Hoomd:

You now have everything you need to run a rigid body simulation, which is what the next tutorial is dedicated, as well has demonstrating how system quantities in a rigid body system are defined. 